# Factor Investing Laboratory - Getting Started

This notebook demonstrates the core functionality of the Factor Investing Laboratory, including:
- Data acquisition from multiple sources
- Factor calculation and analysis
- Portfolio optimization
- Backtesting strategies
- Visualization and reporting

In [28]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import factor lab components
from factor_lab.data import DataManager, YahooFinanceProvider
from factor_lab.factors import FactorCalculator, FactorLibrary
from factor_lab.portfolio import PortfolioOptimizer, PortfolioAnalyzer
from factor_lab.backtesting import Backtester, PerformanceAnalyzer
from factor_lab.visualization import ChartManager, DashboardBuilder
from factor_lab.utils import setup_logging, config

# Setup logging
logger = setup_logging(level="INFO")
print("Factor Investing Laboratory loaded successfully!")

Factor Investing Laboratory loaded successfully!


## 1. Data Acquisition

Let's start by acquiring price data for a universe of stocks.

In [29]:
# Initialize data manager
data_manager = DataManager(primary_provider="yahoo")

# Get a sample universe (subset of S&P 500)
universe = data_manager.get_universe("sp500")[:50]  # First 20 stocks for demo
print(f"Selected universe: {universe}")

# Define date range
start_date = "2022-01-01"
end_date = "2025-05-30"

# Fetch price data
print("Fetching price data...")
prices = data_manager.get_prices(universe, start_date, end_date)
print(f"Price data shape: {prices.shape}")
print(f"Date range: {prices.index[0]} to {prices.index[-1]}")

# Calculate returns
returns = data_manager.get_returns(universe, start_date, end_date)
print(f"Returns data shape: {returns.shape}")

# Display first few rows
print("\nFirst 5 rows of price data:")
print(prices.head())

Selected universe: ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'JPM', 'JNJ', 'V', 'PG', 'UNH', 'HD', 'MA', 'DIS', 'PYPL', 'BAC', 'NFLX', 'ADBE', 'CRM', 'CMCSA', 'XOM', 'VZ', 'KO', 'PEP', 'T', 'ABT', 'COST', 'TMO', 'AVGO', 'ACN', 'CVX', 'NKE', 'MRK', 'DHR', 'LLY', 'TXN', 'NEE', 'WMT', 'QCOM', 'MDT', 'BMY', 'UNP', 'IBM', 'HON', 'AMGN', 'PM', 'RTX', 'LIN', 'LOW']
Fetching price data...
Price data shape: (854, 50)
Date range: 2022-01-03 00:00:00 to 2025-05-29 00:00:00
Returns data shape: (853, 50)

First 5 rows of price data:
Ticker            AAPL         ABT         ACN        ADBE        AMGN  \
Date                                                                     
2022-01-03  178.645645  130.151337  385.630981  564.369995  202.714874   
2022-01-04  176.378357  127.090431  382.875183  554.000000  203.743240   
2022-01-05  171.686722  126.519409  376.132507  514.429993  201.328766   
2022-01-06  168.820679  126.500671  357.968933  514.119995  201.355591   
2022-01-07  16

## 2. Factor Calculation

Now let's calculate various equity factors.

In [30]:
# Initialize factor calculator
factor_calc = FactorCalculator(data_manager)

# Calculate individual factors
print("Calculating factors...")

# Momentum factors
momentum_12m = factor_calc.momentum(prices, lookback=252)
momentum_3m = factor_calc.momentum(prices, lookback=63)

# Mean reversion
short_term_reversal = factor_calc.short_term_reversal(prices, lookback=21)

# Volatility
volatility = factor_calc.volatility(returns, lookback=252)

# Technical indicators
rsi = factor_calc.rsi(prices, period=14)
bb_position = factor_calc.bollinger_position(prices, window=20)

print(f"Momentum (12M) shape: {momentum_12m.shape}")
print(f"Volatility shape: {volatility.shape}")
print(f"RSI shape: {rsi.shape}")

# Calculate all technical factors at once
all_factors = factor_calc.calculate_all_technical_factors(prices, returns)
print(f"\nAll factors shape: {all_factors.shape}")
print(f"Factor names: {list(all_factors.columns.get_level_values(0).unique())}")

Calculating factors...
Momentum (12M) shape: (602, 50)
Volatility shape: (602, 50)
RSI shape: (841, 50)

All factors shape: (849, 550)
Factor names: ['momentum_252d', 'momentum_63d', 'momentum_21d', 'st_reversal_21d', 'st_reversal_5d', 'volatility_252d', 'volatility_63d', 'rsi_14d', 'bb_position_20d', 'price_to_sma_50d', 'price_to_sma_200d']


## 3. Factor Analysis and Normalization

Let's analyze and normalize our factors.

In [31]:
# Extract a specific factor for analysis
momentum_factor = momentum_12m.dropna()

# Normalize factors
momentum_zscore = FactorLibrary.zscore_normalize(momentum_factor)
momentum_rank = FactorLibrary.rank_normalize(momentum_factor)

print("Factor normalization comparison:")
print(f"Original momentum - Mean: {momentum_factor.mean().mean():.4f}, Std: {momentum_factor.std().mean():.4f}")
print(f"Z-score momentum - Mean: {momentum_zscore.mean().mean():.4f}, Std: {momentum_zscore.std().mean():.4f}")
print(f"Rank momentum - Mean: {momentum_rank.mean().mean():.4f}, Std: {momentum_rank.std().mean():.4f}")

# Calculate factor correlation
factor_subset = pd.DataFrame({
    'Momentum_12M': momentum_12m.iloc[-1],
    'Momentum_3M': momentum_3m.iloc[-1],
    'Volatility': volatility.iloc[-1],
    'RSI': rsi.iloc[-1]
}).dropna()

correlation_matrix = FactorLibrary.factor_correlation_matrix(factor_subset)
print("\nFactor correlation matrix:")
print(correlation_matrix.round(3))

Factor normalization comparison:
Original momentum - Mean: 18.6217, Std: 22.3374
Z-score momentum - Mean: 0.0000, Std: 1.0000
Rank momentum - Mean: 0.5100, Std: 0.2116

Factor correlation matrix:
              Momentum_12M  Momentum_3M  Volatility    RSI
Momentum_12M         1.000        0.784       0.210  0.415
Momentum_3M          0.784        1.000       0.243  0.657
Volatility           0.210        0.243       1.000  0.361
RSI                  0.415        0.657       0.361  1.000


## 4. Portfolio Optimization

Let's build portfolios using different optimization techniques.

In [32]:
# Debug: Let's examine the returns data before optimization
print("🔍 Debugging Returns Data")
print("=" * 30)

# Check the returns data quality
print(f"Returns data shape: {returns.shape}")
print(f"Returns data type: {type(returns)}")
print(f"Date range: {returns.index[0]} to {returns.index[-1]}")

# Statistical summary
print(f"\nStatistical Summary:")
print(f"Mean daily return: {returns.mean().mean():.8f} ({returns.mean().mean()*252:.4f} annualized)")
print(f"Daily volatility: {returns.std().mean():.8f} ({returns.std().mean()*np.sqrt(252):.4f} annualized)")
print(f"Min return: {returns.min().min():.6f}")
print(f"Max return: {returns.max().max():.6f}")

# Check for data issues
print(f"\nData Quality:")
print(f"NaN values: {returns.isna().sum().sum()}")
print(f"Infinite values: {np.isinf(returns).sum().sum()}")
print(f"Zero values: {(returns == 0).sum().sum()}")

# Show sample data
print(f"\nSample returns (last 3 days):")
print(returns.tail(3))

# Check individual assets
print(f"\nPer-asset statistics:")
for col in returns.columns[:5]:  # First 5 assets
    asset_returns = returns[col].dropna()
    print(f"{col}: mean={asset_returns.mean():.6f}, std={asset_returns.std():.6f}, count={len(asset_returns)}")

🔍 Debugging Returns Data
Returns data shape: (853, 50)
Returns data type: <class 'pandas.core.frame.DataFrame'>
Date range: 2022-01-04 00:00:00 to 2025-05-29 00:00:00

Statistical Summary:
Mean daily return: 0.00039748 (0.1002 annualized)
Daily volatility: 0.01859176 (0.2951 annualized)
Min return: -0.351166
Max return: 0.244326

Data Quality:
NaN values: 0
Infinite values: 0
Zero values: 127

Sample returns (last 3 days):
Ticker          AAPL       ABT       ACN      ADBE      AMGN      AMZN  \
Date                                                                     
2025-05-27  0.025298  0.012490  0.018897  0.013270  0.028676  0.025026   
2025-05-28  0.001049 -0.006920  0.001775 -0.002106 -0.003757 -0.006310   
2025-05-29 -0.002345  0.006287  0.005507  0.002741  0.018463  0.004787   

Ticker          AVGO       BAC       BMY     CMCSA  ...         T       TMO  \
Date                                                ...                       
2025-05-27  0.030299  0.023611  0.000854  0.

In [33]:
# Prepare data for optimization (use more data for stability)
recent_returns = returns.tail(504)  # Last 2 years of data for better estimation
print(f"Using returns data from {recent_returns.index[0]} to {recent_returns.index[-1]}")
print(f"Returns data shape: {recent_returns.shape}")
print(f"Number of observations: {len(recent_returns)}")

# Check for any issues with the data
print(f"\nData quality check:")
print(f"Any NaN values: {recent_returns.isna().sum().sum()}")
print(f"All finite values: {np.isfinite(recent_returns).all().all()}")

# Remove any assets with insufficient data
min_observations = 200
valid_assets = recent_returns.dropna(axis=1, thresh=min_observations)
print(f"Assets with sufficient data: {valid_assets.shape[1]} out of {recent_returns.shape[1]}")

if valid_assets.shape[1] < recent_returns.shape[1]:
    print(f"Dropping assets with insufficient data...")
    recent_returns = valid_assets

# Initialize portfolio optimizer
print(f"\nInitializing optimizer with {recent_returns.shape[1]} assets...")
optimizer = PortfolioOptimizer(recent_returns)

# Check covariance matrix condition
cov_eigenvals = np.linalg.eigvals(optimizer.cov_matrix.values)
print(f"Covariance matrix condition number: {np.max(cov_eigenvals) / np.min(cov_eigenvals):.2e}")
print(f"Minimum eigenvalue: {np.min(cov_eigenvals):.2e}")

# Mean-variance optimization
print("\nRunning mean-variance optimization...")
mv_result = optimizer.mean_variance_optimization()
print(f"Optimization method used: {mv_result.get('method', 'unknown')}")
if 'error' in mv_result:
    print(f"Warning: {mv_result['error']}")
print(f"Expected return: {mv_result['expected_return']:.4f}")
print(f"Volatility: {mv_result['volatility']:.4f}")
print(f"Sharpe ratio: {mv_result['sharpe_ratio']:.4f}")

# Risk parity optimization
print("\nRunning risk parity optimization...")
rp_result = optimizer.risk_parity_optimization()
print(f"Expected return: {rp_result['expected_return']:.4f}")
print(f"Volatility: {rp_result['volatility']:.4f}")
print(f"Sharpe ratio: {rp_result['sharpe_ratio']:.4f}")

# Minimum variance optimization
print("\nRunning minimum variance optimization...")
minvol_result = optimizer.minimum_variance_optimization()
print(f"Expected return: {minvol_result['expected_return']:.4f}")
print(f"Volatility: {minvol_result['volatility']:.4f}")
print(f"Sharpe ratio: {minvol_result['sharpe_ratio']:.4f}")

# Show top holdings for mean-variance portfolio
mv_weights = pd.Series(mv_result['weights']).sort_values(ascending=False)
print("\nTop 5 holdings (Mean-Variance):")
print(mv_weights.head())

2025-05-31 15:23:38,238 - factor_lab.portfolio - INFO - Returns data stats - Mean: 0.000787, Std: 0.017207, Min: -0.223797, Max: 0.244326
2025-05-31 15:23:38,242 - factor_lab.portfolio - WARNING - Found NaN values in expected returns. Replacing with 0.
2025-05-31 15:23:38,243 - factor_lab.portfolio - INFO - Expected returns range: [-0.805386, 0.208695]
2025-05-31 15:23:38,242 - factor_lab.portfolio - WARNING - Found NaN values in expected returns. Replacing with 0.
2025-05-31 15:23:38,243 - factor_lab.portfolio - INFO - Expected returns range: [-0.805386, 0.208695]
2025-05-31 15:23:38,249 - factor_lab.portfolio - WARNING - Shrinkage covariance failed: Input contains infinity or a value too large for dtype('float64').. Using sample covariance.
2025-05-31 15:23:38,249 - factor_lab.portfolio - WARNING - Shrinkage covariance failed: Input contains infinity or a value too large for dtype('float64').. Using sample covariance.


Using returns data from 2023-05-25 00:00:00 to 2025-05-29 00:00:00
Returns data shape: (504, 50)
Number of observations: 504

Data quality check:
Any NaN values: 0
All finite values: True
Assets with sufficient data: 50 out of 50

Initializing optimizer with 50 assets...


2025-05-31 15:23:38,258 - factor_lab.portfolio - INFO - Covariance matrix is positive definite.
2025-05-31 15:23:38,266 - factor_lab.portfolio - INFO - Covariance matrix condition number: 1.80e+02
2025-05-31 15:23:38,267 - factor_lab.portfolio - INFO - Eigenvalue range: [7.71e+03, 1.39e+06]
2025-05-31 15:23:38,268 - factor_lab.portfolio - INFO - Attempting mean-variance optimization with OSQP solver
2025-05-31 15:23:38,266 - factor_lab.portfolio - INFO - Covariance matrix condition number: 1.80e+02
2025-05-31 15:23:38,267 - factor_lab.portfolio - INFO - Eigenvalue range: [7.71e+03, 1.39e+06]
2025-05-31 15:23:38,268 - factor_lab.portfolio - INFO - Attempting mean-variance optimization with OSQP solver


Covariance matrix condition number: 1.80e+02
Minimum eigenvalue: 7.71e+03

Running mean-variance optimization...


2025-05-31 15:23:38,282 - factor_lab.portfolio - INFO - Mean-variance optimization successful with OSQP solver


Optimization method used: mean_variance
Expected return: 0.0769
Volatility: 99.2322
Sharpe ratio: 0.0008

Running risk parity optimization...


2025-05-31 15:23:38,483 - factor_lab.portfolio - INFO - Attempting minimum variance optimization with OSQP solver


Expected return: -0.0705
Volatility: 30.9041
Sharpe ratio: -0.0023

Running minimum variance optimization...


2025-05-31 15:23:38,491 - factor_lab.portfolio - INFO - Minimum variance optimization successful with OSQP


Expected return: -0.0500
Volatility: 24.1355
Sharpe ratio: -0.0021

Top 5 holdings (Mean-Variance):
V       0.32780
BMY     0.12144
LLY     0.08072
AAPL    0.08052
NFLX    0.05354
dtype: float64


## 5. Strategy Definition and Backtesting

Let's define a simple momentum strategy and backtest it.

In [34]:
def momentum_strategy(factor_data, price_data, top_n=5, **kwargs):
    """
    Simple momentum strategy: go long top momentum stocks.
    """
    try:
        # Get latest factor scores
        latest_factors = factor_data.iloc[-1]
        
        # Select top momentum stocks
        top_stocks = latest_factors.nlargest(top_n)
        
        # Equal weight the selected stocks
        weights = pd.Series(index=latest_factors.index, data=0.0)
        weights[top_stocks.index] = 1.0 / len(top_stocks)
        
        return weights
    except Exception as e:
        print(f"Error in momentum strategy: {e}")
        return None

# Prepare factor data for backtesting
momentum_for_bt = momentum_12m.copy()

# Initialize backtester
backtester = Backtester(
    start_date=start_date,
    end_date=end_date,
    initial_capital=100000,
    transaction_cost=0.001,
    rebalance_frequency='monthly'
)

print("Running momentum strategy backtest...")
backtest_results = backtester.run_factor_strategy_backtest(
    price_data=prices,
    factor_data=momentum_for_bt,
    strategy_func=momentum_strategy,
    lookback_window=252,
    top_n=10
)

# Display results
print("\nBacktest Results:")
for metric, value in backtest_results['performance_metrics'].items():
    if isinstance(value, float):
        print(f"{metric}: {value:.4f}")
    else:
        print(f"{metric}: {value}")

print(f"\nPortfolio value time series shape: {backtest_results['portfolio_values'].shape}")
print(f"Final portfolio value: ${backtest_results['portfolio_values'].iloc[-1]:,.2f}")

Running momentum strategy backtest...

Backtest Results:
Total Return: 1.1447
Annualized Return: 0.3907
Annualized Volatility: 0.2118
Sharpe Ratio: 1.8450
Calmar Ratio: 1.8987
Max Drawdown: -0.2058
VaR 95%: -0.0204
VaR 99%: -0.0304
Skewness: 0.2210
Kurtosis: 5.6547
Win Rate: 0.5678
Average Win: 0.0099
Average Loss: -0.0098

Portfolio value time series shape: (583,)
Final portfolio value: $212,121.21

Backtest Results:
Total Return: 1.1447
Annualized Return: 0.3907
Annualized Volatility: 0.2118
Sharpe Ratio: 1.8450
Calmar Ratio: 1.8987
Max Drawdown: -0.2058
VaR 95%: -0.0204
VaR 99%: -0.0304
Skewness: 0.2210
Kurtosis: 5.6547
Win Rate: 0.5678
Average Win: 0.0099
Average Loss: -0.0098

Portfolio value time series shape: (583,)
Final portfolio value: $212,121.21


## 6. Performance Analysis

Let's analyze the strategy performance in detail.

In [35]:
# Initialize performance analyzer
perf_analyzer = PerformanceAnalyzer()

# Calculate portfolio returns
portfolio_analyzer = PortfolioAnalyzer()
portfolio_values = backtest_results['portfolio_values']
portfolio_returns = portfolio_values.pct_change().dropna()

# Calculate comprehensive performance metrics
detailed_metrics = portfolio_analyzer.performance_metrics(portfolio_returns)

print("Detailed Performance Analysis:")
print("=" * 40)
for metric, value in detailed_metrics.items():
    if isinstance(value, float):
        if 'Return' in metric or 'Volatility' in metric:
            print(f"{metric}: {value*100:.2f}%")
        else:
            print(f"{metric}: {value:.4f}")
    else:
        print(f"{metric}: {value}")

# Rolling performance analysis
rolling_perf = perf_analyzer.rolling_performance_analysis(portfolio_values, window=63)
print(f"\nRolling performance metrics calculated for {len(rolling_perf)} periods")

# Latest rolling metrics
if len(rolling_perf) > 0:
    latest_rolling = rolling_perf.iloc[-1]
    print("\nLatest Rolling Metrics (3-month window):")
    print(f"Rolling Return: {latest_rolling['Rolling_Return']*100:.2f}%")
    print(f"Rolling Volatility: {latest_rolling['Rolling_Volatility']*100:.2f}%")
    print(f"Rolling Sharpe: {latest_rolling['Rolling_Sharpe']:.2f}")

Detailed Performance Analysis:
Total Return: 113.74%
Annualized Return: 38.94%
Annualized Volatility: 21.19%
Sharpe Ratio: 1.7435
Max Drawdown: -0.2237
VaR (95%): -0.0204
Skewness: 0.2183
Kurtosis: 5.6529

Rolling performance metrics calculated for 458 periods

Latest Rolling Metrics (3-month window):
Rolling Return: 42.14%
Rolling Volatility: 32.89%
Rolling Sharpe: 1.28


## 7. Visualization

Let's create visualizations to better understand our results.

In [36]:
# Initialize visualization components
chart_manager = ChartManager()
dashboard_builder = DashboardBuilder()

# Create performance dashboard
print("Creating performance visualizations...")

# 1. Cumulative returns chart
cum_returns_fig = chart_manager.plot_cumulative_returns(
    portfolio_returns, 
    title="Momentum Strategy - Cumulative Returns"
)
cum_returns_fig.show()

# 2. Drawdown chart
drawdown_fig = chart_manager.plot_drawdown(
    portfolio_returns,
    title="Momentum Strategy - Drawdown Analysis"
)
drawdown_fig.show()

# 3. Rolling metrics
rolling_fig = chart_manager.plot_rolling_metrics(
    portfolio_returns,
    window=63,
    title="Momentum Strategy - Rolling Performance"
)
rolling_fig.show()

# 4. Portfolio composition (latest)
if 'positions' in backtest_results and len(backtest_results['positions']) > 0:
    latest_positions = backtest_results['positions'].iloc[-1]
    latest_weights = latest_positions.drop('Portfolio_Value', errors='ignore')
    latest_weights = latest_weights[latest_weights > 0.001]  # Filter small weights
    
    composition_fig = chart_manager.plot_portfolio_composition(
        latest_weights.to_dict(),
        title="Current Portfolio Composition"
    )
    composition_fig.show()

print("Visualizations created successfully!")

Creating performance visualizations...


Visualizations created successfully!


## 8. Factor Analysis Visualization

Let's visualize factor characteristics and relationships.

In [37]:
# Prepare factor data for visualization
factor_for_viz = pd.DataFrame({
    'Momentum_12M': momentum_12m.iloc[-1],
    'Momentum_3M': momentum_3m.iloc[-1], 
    'Volatility': volatility.iloc[-1],
    'RSI': rsi.iloc[-1],
    'BB_Position': bb_position.iloc[-1]
}).dropna()

# Factor correlation heatmap
correlation_fig = chart_manager.plot_factor_heatmap(
    factor_for_viz,
    title="Factor Correlation Matrix"
)
correlation_fig.show()

# Factor returns scatter plot (if we have forward returns)
try:
    # Calculate 1-month forward returns for factor analysis
    forward_returns_1m = returns.shift(-21)  # 21 days forward
    
    factor_returns_fig = chart_manager.plot_factor_returns(
        momentum_12m.T,  # Transpose to match expected format
        forward_returns_1m,
        momentum_12m.index[-1]  # Use latest date as factor name
    )
    factor_returns_fig.show()
except Exception as e:
    print(f"Could not create factor returns plot: {e}")

print("Factor analysis visualizations completed!")

Factor analysis visualizations completed!


## 9. Comparison with Benchmark

Let's compare our strategy with a simple buy-and-hold benchmark.

In [38]:
# Create a simple equal-weight benchmark
equal_weight_returns = returns.mean(axis=1)  # Equal weight all stocks
benchmark_returns = equal_weight_returns.loc[portfolio_returns.index]

# Compare strategies
strategy_comparison = {
    'Momentum Strategy': {
        'performance_metrics': detailed_metrics
    },
    'Equal Weight Benchmark': {
        'performance_metrics': portfolio_analyzer.performance_metrics(benchmark_returns)
    }
}

comparison_df = perf_analyzer.compare_strategies(strategy_comparison)
print("Strategy Comparison:")
print("=" * 50)
print(comparison_df.round(4))

# Benchmark comparison analysis
benchmark_analysis = perf_analyzer.benchmark_comparison(portfolio_returns, benchmark_returns)
print("\nBenchmark Comparison Metrics:")
print("=" * 35)
for metric, value in benchmark_analysis.items():
    if isinstance(value, float):
        print(f"{metric}: {value:.4f}")

# Plot comparison
comparison_fig = chart_manager.plot_cumulative_returns(
    portfolio_returns,
    benchmark_returns,
    title="Strategy vs Benchmark - Cumulative Returns"
)
comparison_fig.show()

print("\nBenchmark comparison completed!")

Strategy Comparison:
                        Total Return  Annualized Return  \
Momentum Strategy             1.1374             0.3894   
Equal Weight Benchmark        0.4545             0.1761   

                        Annualized Volatility  Sharpe Ratio  Max Drawdown  \
Momentum Strategy                      0.2119        1.7435       -0.2237   
Equal Weight Benchmark                 0.1366        1.1432       -0.1641   

                        VaR (95%)  Skewness  Kurtosis  
Momentum Strategy         -0.0204    0.2183    5.6529  
Equal Weight Benchmark    -0.0121    0.4567   14.9879  

Benchmark Comparison Metrics:
Alpha (Daily): 0.0006
Beta: 1.1656
Tracking Error: 0.1417
Information Ratio: 1.2699
Up Capture: 1.2725
Down Capture: 1.0743
Active Return: 0.1799



Benchmark comparison completed!


## 10. Export Results

Let's save our results for future analysis.

In [39]:
import os
from factor_lab.utils import FileUtils

# Create results directory
results_dir = "../results"
FileUtils.ensure_directory(results_dir)

# Save backtest results
portfolio_values.to_csv(f"{results_dir}/momentum_strategy_portfolio_values.csv")
backtest_results['positions'].to_csv(f"{results_dir}/momentum_strategy_positions.csv")

# Save factor data
momentum_12m.to_csv(f"{results_dir}/momentum_12m_factor.csv")
factor_for_viz.to_csv(f"{results_dir}/factor_cross_section.csv")

# Save performance metrics
metrics_df = pd.DataFrame.from_dict(detailed_metrics, orient='index', columns=['Value'])
metrics_df.to_csv(f"{results_dir}/performance_metrics.csv")

# Save comparison
comparison_df.to_csv(f"{results_dir}/strategy_comparison.csv")

print(f"Results saved to {results_dir}/")
print("Files created:")
for file in os.listdir(results_dir):
    print(f"  - {file}")

Results saved to ../results/
Files created:
  - momentum_12m_factor.csv
  - momentum_strategy_positions.csv
  - factor_cross_section.csv
  - momentum_strategy_portfolio_values.csv
  - strategy_comparison.csv
  - performance_metrics.csv


## Summary

This notebook demonstrated the core capabilities of the Factor Investing Laboratory:

1. **Data Acquisition**: Fetched price and returns data from Yahoo Finance
2. **Factor Calculation**: Computed various technical factors (momentum, volatility, RSI, etc.)
3. **Factor Analysis**: Normalized factors and analyzed correlations
4. **Portfolio Optimization**: Built portfolios using different optimization techniques
5. **Strategy Backtesting**: Implemented and backtested a momentum strategy
6. **Performance Analysis**: Conducted comprehensive performance evaluation
7. **Visualization**: Created interactive charts and dashboards
8. **Benchmark Comparison**: Compared strategy performance against benchmarks
9. **Results Export**: Saved results for future analysis

### Next Steps

- Experiment with different factor combinations
- Try alternative optimization methods
- Implement more sophisticated strategies
- Add fundamental factors using OpenBB data
- Explore multi-factor models
- Build custom dashboards for ongoing monitoring

The Factor Investing Laboratory provides a comprehensive framework for factor research and strategy development!